<a href="https://colab.research.google.com/github/vishnudas-raveendran/PGP-AIML/blob/master/NLP/Project1/NLP_Project_1_Part_B_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part B: Bot

In [102]:
import json
import pandas as pd
import numpy as np
import pickle

In [75]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
with open('GL Bot.json') as file:
  data = json.load(file)

In [10]:
tagPatterns = []

In [12]:
for intent in data['intents']:
  tag = intent['tag']
  print(tag)
  for pattern in intent['patterns']:
    tagPatterns.append([pattern, tag])

Intro
Exit
Olympus
SL
NN
Bot
Profane
Ticket


In [13]:
tagPatterns

[['hi', 'Intro'],
 ['how are you', 'Intro'],
 ['is anyone there', 'Intro'],
 ['hello', 'Intro'],
 ['whats up', 'Intro'],
 ['hey', 'Intro'],
 ['yo', 'Intro'],
 ['listen', 'Intro'],
 ['please help me', 'Intro'],
 ['i am learner from', 'Intro'],
 ['i belong to', 'Intro'],
 ['aiml batch', 'Intro'],
 ['aifl batch', 'Intro'],
 ['i am from', 'Intro'],
 ['my pm is', 'Intro'],
 ['blended', 'Intro'],
 ['online', 'Intro'],
 ['i am from', 'Intro'],
 ['hey ya', 'Intro'],
 ['talking to you for first time', 'Intro'],
 ['thank you', 'Exit'],
 ['thanks', 'Exit'],
 ['cya', 'Exit'],
 ['see you', 'Exit'],
 ['later', 'Exit'],
 ['see you later', 'Exit'],
 ['goodbye', 'Exit'],
 ['i am leaving', 'Exit'],
 ['have a Good day', 'Exit'],
 ['you helped me', 'Exit'],
 ['thanks a lot', 'Exit'],
 ['thanks a ton', 'Exit'],
 ['you are the best', 'Exit'],
 ['great help', 'Exit'],
 ['too good', 'Exit'],
 ['you are a good learning buddy', 'Exit'],
 ['olympus', 'Olympus'],
 ['explain me how olympus works', 'Olympus'],
 ['I

In [28]:
intents = pd.DataFrame(tagPatterns, columns =['pattern', 'tag'])

In [29]:
intents.head()

,pattern,tag
0,hi,Intro
1,how are you,Intro
2,is anyone there,Intro
3,hello,Intro
4,whats up,Intro


In [18]:
#Save the intents as csv
intents.to_csv('intents.csv')

In [99]:
responses = {}

In [108]:
for intent in data['intents']:
  tag = intent['tag']
  for response in intent['responses']:
    responses[tag.lower()]=response               #In given JSON, only one response is given so we can store it in dictionary for O(1) lookup later 

In [109]:
with open('responses.pkl', 'wb') as f:
    pickle.dump(responses, f)

with open('responses.pkl', 'rb') as f:
    responses = pickle.load(f)

In [110]:
def get_response_for_tag(tag):
  if tag.lower() in responses.keys():
    return responses[tag.lower()]
  else:
    return "Can you please rephrase that perhaps?"

In [111]:
get_response_for_tag("olympus")

'Link: Olympus wiki'

## 2. Clean the inputs

In [21]:
!pip install contractions
import nltk
import inflect
import contractions
from bs4 import BeautifulSoup
import re, string, unicodedata
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

     |████████████████████████████████| 284 kB 30.3 MB/s 
     |████████████████████████████████| 106 kB 60.8 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [20]:
# Pipeline for text cleaning

def denoise_text(text):
    # Strip html if any. For ex. removing <html>, <p> tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    # Replace contractions in the text. For ex. didn't -> did not
    text = contractions.fix(text)
    return text

def tokenize(text):
    return nltk.word_tokenize(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
def replace_numbers(words):
    """Replace all integer occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
def remove_numbers(words):
    """Remove all integer occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = ''
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [27]:
def normalize_text(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = remove_numbers(words)
    #words = remove_stopwords(words)
    #words = stem_words(words)
    words = lemmatize_verbs(words)
    return words

In [30]:
def text_clean(text):
    text = denoise_text(text)
    text = ' '.join([x for x in normalize_text(tokenize(text))])
    return text
intents['pattern'] = [text_clean(x) for x in intents['pattern']]

In [25]:
intents.shape

(128, 2)

In [31]:
intents.head()

,pattern,tag
0,hi,Intro
1,how be you,Intro
2,be anyone there,Intro
3,hello,Intro
4,what be up,Intro


### 3. Train-Test Split Data

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test =train_test_split(intents.pattern,intents.tag, random_state=42,test_size = 0.3,shuffle = True)

In [34]:
print(f"Training Size:{y_train.shape[0]} \n Test Size: {y_test.shape[0]}")

Training Size:89 
 Test Size: 39


### 4. Vectorize the inputs:

We can use Tf-IDF here

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvectorizer = TfidfVectorizer(min_df=1, max_features=1000)
tfidf_X = tfidfvectorizer.fit_transform(X_train)
tfidfvectorizer.get_feature_names_out()

array(['able', 'ada', 'adam', 'an', 'ann', 'anyone', 'artificial', 'bad',
       'bag', 'bay', 'be', 'best', 'boost', 'buddy', 'classification',
       'contact', 'create', 'cya', 'day', 'deep', 'diffult', 'do',
       'ensemble', 'epoch', 'epochs', 'explain', 'for', 'forest',
       'forward', 'from', 'good', 'gradient', 'great', 'hate', 'have',
       'hell', 'hello', 'help', 'hey', 'hide', 'hours', 'how', 'in',
       'intelligence', 'jerk', 'joke', 'knn', 'later', 'layer', 'learn',
       'learner', 'link', 'listen', 'logistic', 'lot', 'machine', 'me',
       'ml', 'my', 'naive', 'name', 'net', 'network', 'neural', 'no',
       'not', 'of', 'olympus', 'online', 'operation', 'please', 'pm',
       'problem', 'problems', 'propagation', 'random', 'regression',
       'relu', 'see', 'sgd', 'sigmoid', 'sl', 'smart', 'softmax',
       'solution', 'solve', 'stupid', 'svm', 'teach', 'techbniques',
       'techniques', 'thank', 'the', 'there', 'think', 'ticket', 'to',
       'too', 'unable'

In [36]:
tfidf_X.toarray()
tfidf_X_train = pd.DataFrame(tfidf_X.todense(), columns = tfidfvectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### 5. Prepare Model and Train

In [38]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_le = le.fit_transform(y_train)

In [133]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
clf = OneVsRestClassifier(SVC(random_state=42)).fit(tfidf_X_train, y_train_le)

### 6. Evaluate Trained Model

In [41]:
tfidf_X_test = tfidfvectorizer.transform(X_test)
tfidf_X_test.toarray()
tfidf_X_test = pd.DataFrame(tfidf_X_test.todense(), columns = tfidfvectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [44]:
predicted = clf.predict(tfidf_X_test)
predicted_labels = le.inverse_transform(predicted)

In [45]:
y_test_le = le.transform(y_test)

In [53]:
from sklearn.metrics import accuracy_score,  f1_score
def print_report(modelName,predicted_labels):
  print("Model: "+modelName)
  accuracy = accuracy_score(y_test, predicted_labels)
  print(f"Accuracy: {accuracy}")
  f1= f1_score(y_test, predicted_labels,average="macro")
  print(f"F1 Score: {f1}")
  #r = [modelName,accuracy,f1]
  #report.append(r)

### 7. Evaluation Report of Trained Model

In [54]:
print_report("tfidf",predicted_labels)

Model: tfidf
Accuracy: 0.5128205128205128
F1 Score: 0.45968614718614714


### 8. Create Preprocessing pipeline for input from chat

In [65]:
def preprocess_input(text):
  text = text_clean(text)
  text_vectors = tfidfvectorizer.transform([text])
  text_vectors.toarray()
  text_vectors = pd.DataFrame(text_vectors.todense(), columns = tfidfvectorizer.get_feature_names())
  return text_vectors


In [123]:
def get_tag(text):
  text_vectors = preprocess_input(text)
  prediction = clf.predict(text_vectors)
  tag = le.inverse_transform(prediction)
  return tag[0]

### 9. Test Sample Inputs

In [116]:
get_tag("what is your name")

'Bot'

In [117]:
get_tag("Thanks a ton")

'Exit'

In [118]:
get_tag("What is deep learning")

'NN'

In [119]:
get_tag("See you")

'Exit'

In [120]:
get_tag("Bye")   ##wrong -- have to improve model or manually tune response

'NN'

In [121]:
get_tag("Not helped") 

'Ticket'

In [122]:
get_tag("Having trouble accessing olympus")

'Olympus'

### Put all components together

In [140]:
def manual_corrections(text):
  if text.lower() == "bye":
    return "Exit", True
  elif text.lower() == "hi":
    return "Intro",True
  else:
    return "",False

In [126]:
def ask_bot(text):
  tag = get_tag(text)
  tag_manual, override = manual_corrections(text)
  if override:
    tag = tag_manual
  response = get_response_for_tag(tag)
  return response

### Test the backend

In [128]:
ask_bot("Hi, how are you?")

'Hello! how can i help you ?'

In [129]:
ask_bot("Bye")  

'I hope I was able to assist you, Good Bye'

In [132]:
ask_bot("Does god exists?")

'Tarnsferring the request to your PM'

In [141]:
def chat_window():
  try:
    print("Welcome to Great Learning !")
    print("How can I help you today?")
    while True:
      inp = input("\n\n >>")
      response = ask_bot(inp.lower())
      print(response)
      if response == "I hope I was able to assist you, Good Bye":
        break
  except KeyboardInterrupt:
    print("I hope I was able to assist you, Good Bye")


In [142]:
chat_window()

Welcome to Great Learning !
How can I help you today?


 >>Hi
Hello! how can i help you ?


 >>how are you?
Hello! how can i help you ?


 >>what is deep learning?
Link: Neural Nets wiki


 >>Cannot access olympus
Link: Olympus wiki


 >>what are some ensemble techniques
Link: Machine Learning wiki 


 >>who are youi
I am your virtual learning assistant


 >>There is some problem
Hello! how can i help you ?


 >>can you create a ticket?
Tarnsferring the request to your PM


 >>Thank you
I hope I was able to assist you, Good Bye
